In [2]:
import great_expectations as ge
import pandas as pd
import logging
from typing import Dict, Tuple, Any

/Users/maxwell.marcos/.pyenv/versions/3.9.6/envs/venv-mlops/lib/python3.9/site-packages/great_expectations/compatibility/sqlalchemy.py:20: UserWarning: SQLAlchemy v2.0.0 or later is not yet supported by Great Expectations.
  warnings.warn(


In [23]:
df = pd.read_csv("../data/01_raw/house-pricing.csv")
df_model_input = pd.read_csv("../data/05_model_input/X_train_transformed.csv")

In [4]:
df.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
gdf = ge.from_pandas(df)

In [6]:
gdf.expect_column_values_to_not_be_null("Id")


{
  "meta": {},
  "result": {
    "element_count": 1460,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [7]:
gdf.validate()

{
  "meta": {
    "great_expectations_version": "0.16.14",
    "expectation_suite_name": "default",
    "run_id": {
      "run_name": null,
      "run_time": "2023-05-29T21:09:11.026652+01:00"
    },
    "batch_kwargs": {
      "ge_batch_id": "abdee838-fe5c-11ed-b5c2-acde48001122"
    },
    "batch_markers": {},
    "batch_parameters": {},
    "validation_time": "20230529T200911.026564Z",
    "expectation_suite_meta": {
      "great_expectations_version": "0.16.14"
    }
  },
  "statistics": {
    "evaluated_expectations": 1,
    "successful_expectations": 1,
    "unsuccessful_expectations": 0,
    "success_percent": 100.0
  },
  "evaluation_parameters": {},
  "success": true,
  "results": [
    {
      "meta": {},
      "result": {
        "element_count": 1460,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "partial_unexpected_list": []
      },
      "success": true,
      "expectation_config": {
        "kwargs": {

In [9]:
def check_nulls(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict]:
    """Check for nulls in the dataframe.
    Args:
    --
        df (pd.DataFrame): Dataframe to check for nulls.
    Returns:
    --
        df (pd.DataFrame): Dataframe with nulls removed.
        describe_to_dict (dict): Description of the dataframe.
    """
    
    gdf = ge.from_pandas(df)
    gdf.expect_column_values_to_not_be_null("Id")
    return gdf

In [33]:
import yaml
with open('../conf/base/parameters/data_quality.yml') as f:
    parameters = yaml.load(f, Loader=yaml.loader.SafeLoader)

logger = logging.getLogger(__name__)

def check_ranges(df: pd.DataFrame, parameters : Dict[str, Any]) -> Tuple[pd.DataFrame, Dict]:
    """Check for set of itens in categorcial variables.
    Args:
    --
        df (pd.DataFrame): Dataframe to check for nulls.
    Returns:
    --
        df (pd.DataFrame): Dataframe with nulls removed.
        describe_to_dict (dict): Description of the dataframe.
    """
    
    num_cols = df.select_dtypes(include=['number']).columns
    ranges = parameters["num_quality_ranges"]
    gdf = ge.from_pandas(df)
    for column in num_cols:
        gdf.expect_column_values_to_be_between(column,ranges['min'],ranges['max'])
    
    validation_results = gdf.validate()
    failed_expectations = [result for result in validation_results["results"] if not result["success"]]
    
    logger.info(
        f"Total Expectations: {len(validation_results['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
    if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"\nFailed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )
   
    return df

In [34]:
check_ranges(df_model_input, parameters)

Exception: Data Quality Validation Failed: ["\nFailed Expectation 1:  Expectation Type: expect_column_values_to_be_between  Column: numerical__LotFrontage  Details: {'element_count': 1168, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 5, 'unexpected_percent': 0.4280821917808219, 'unexpected_percent_total': 0.4280821917808219, 'unexpected_percent_nonmissing': 0.4280821917808219, 'partial_unexpected_list': [12.77137639050307, 5.1397974431346505, 12.77137639050307, 5.876639548397808, 5.455586916818861]}", "\nFailed Expectation 2:  Expectation Type: expect_column_values_to_be_between  Column: numerical__LotArea  Details: {'element_count': 1168, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 18, 'unexpected_percent': 1.5410958904109588, 'unexpected_percent_total': 1.5410958904109588, 'unexpected_percent_nonmissing': 1.5410958904109588, 'partial_unexpected_list': [5.559054160841287, 8.99373898243268, 5.390553765728527, 7.414503677587988, 25.66385022187101, 